This script was used to compare results considering the different techniques used between group 1 (% use in winter) and group 2 (kshape clustering)

In [110]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import time

In [111]:
# define data location
# location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'

In [121]:
# import data

### group 1 data 
group_1 = pd.read_pickle(location+'codebook_full_dmeterno_daily.pkl.zip')
group_1.head()

,ID,TARIFF,REVENUCODE,Use,UseGroup,ColdGroup,DayGroup,Cluster
0,606395738288,GH0,1.0,1.356354,0,1,0,N_M
1,608923719096,GH0,1.0,0.582873,0,2,0,N_H
2,640944705832,GH0,1.0,2.998011,1,2,0,L_H
3,726789359116,GH0,1.0,3.856354,2,2,0,M_H
4,640143121596,UHC,3.0,5.585331,2,0,0,M_L


In [113]:
### group 2 data

# sufficient data
#group_2_suffi_R = pd.read_excel(location+'suffi_R.xlsx')
#group_2_suffi_C = pd.read_excel(location+'suffi_C.xlsx')

# winter clusters
group_2_RW = pd.read_excel(location+'test_RW.xlsx')
group_2_CW = pd.read_excel(location+'test_CW.xlsx')

# full year
#group_2_R = pd.read_excel(location+'test_R.xlsx')
#group_2_C = pd.read_excel(location+'test_C.xlsx')

In [114]:
# keep only relevant cols
group_1 = group_1[['ID','TARIFF','REVENUCODE','Use','UseGroup','ColdGroup','Cluster']]
group_2_RW = group_2_RW[['DMeterNo','AVG','Lable']]
group_2_CW = group_2_CW[['DMeterNo','AVG','Lable','Size']]

In [115]:
# remove duplicates
group_1 = group_1.drop_duplicates("ID") 
group_2_RW = group_2_RW.drop_duplicates("DMeterNo") 
group_2_CW = group_2_CW.drop_duplicates("DMeterNo") 

In [116]:
# define cluster labels for group 2
group_2_RW['Cluster'] = None
group_2_RW.loc[(group_2_RW.Lable == 0), 'Cluster'] = 'R_0' 
group_2_RW.loc[(group_2_RW.Lable == 1), 'Cluster'] = 'R_1'
group_2_RW.loc[(group_2_RW.Lable == 2), 'Cluster'] = 'R_2'

group_2_RW = group_2_RW.drop(columns=['Lable'])

group_2_CW['Cluster'] = None
group_2_CW.loc[ (group_2_CW.Lable == 1) & (group_2_CW.Size == 'S'), 'Cluster'] = 'C_S_1' 
group_2_CW.loc[ (group_2_CW.Lable == 2) & (group_2_CW.Size == 'S'), 'Cluster'] = 'C_S_2'
group_2_CW.loc[ (group_2_CW.Lable == 3) & (group_2_CW.Size == 'S'), 'Cluster'] = 'C_S_3'
group_2_CW.loc[ (group_2_CW.Lable == 4) & (group_2_CW.Size == 'S'), 'Cluster'] = 'C_S_4'

group_2_CW.loc[ (group_2_CW.Lable == 1) & (group_2_CW.Size == 'M'), 'Cluster'] = 'C_M_1' 
group_2_CW.loc[ (group_2_CW.Lable == 2) & (group_2_CW.Size == 'M'), 'Cluster'] = 'C_M_2'
group_2_CW.loc[ (group_2_CW.Lable == 3) & (group_2_CW.Size == 'M'), 'Cluster'] = 'C_M_3'
group_2_CW.loc[ (group_2_CW.Lable == 4) & (group_2_CW.Size == 'M'), 'Cluster'] = 'C_M_4'

group_2_CW.loc[ (group_2_CW.Lable == 1) & (group_2_CW.Size == 'L'), 'Cluster'] = 'C_L_1' 
group_2_CW.loc[ (group_2_CW.Lable == 2) & (group_2_CW.Size == 'L'), 'Cluster'] = 'C_L_2'
group_2_CW.loc[ (group_2_CW.Lable == 3) & (group_2_CW.Size == 'L'), 'Cluster'] = 'C_L_3'
group_2_CW.loc[ (group_2_CW.Lable == 4) & (group_2_CW.Size == 'L'), 'Cluster'] = 'C_L_4'

group_2_CW = group_2_CW.drop(columns=['Lable','Size'])


In [117]:
# combine group2
group2list = [group_2_RW, group_2_CW]
group_2 = pd.concat(group2list)

In [118]:
print(group_1['ID'].nunique(), group_2['DMeterNo'].nunique(), group_1['ID'].nunique()/group_2['DMeterNo'].nunique())
# we have 97% overlap for accounts... not sure why the difference

6137 6293 0.9752105514063245


In [119]:
# merge dfs
group_1['ID'] = group_1['ID'].astype(int) 
group_1.rename(columns={'Cluster':'Cluster1'}, inplace=True)

group_2.rename(columns={'Cluster':'Cluster2'}, inplace=True)

clusters = group_1.merge(group_2, how='inner', left_on=['ID'], right_on=['DMeterNo'])


In [120]:
clusters.head()

,ID,TARIFF,REVENUCODE,Use,UseGroup,ColdGroup,Cluster1,DMeterNo,AVG,Cluster2
0,606395738288,GH0,1.0,1.356354,0,1,N_M,606395738288,2.142857,R_0
1,608923719096,GH0,1.0,0.582873,0,2,N_H,608923719096,1.277311,R_0
2,640944705832,GH0,1.0,2.998011,1,2,L_H,640944705832,5.364706,R_0
3,726789359116,GH0,1.0,3.856354,2,2,M_H,726789359116,7.378151,R_0
4,640143121596,UHC,3.0,5.585331,2,0,M_L,640143121596,6.454667,C_S_2


In [100]:
pd.crosstab(clusters.Cluster1,clusters.Cluster2)

Cluster2,C_L_1,C_L_2,C_M_1,C_M_2,C_M_3,C_S_1,C_S_2,C_S_3,C_S_4,R_0,R_1,R_2
Cluster1,,,,,,,,,,,,
H_H,0,0,34,4,1,183,17,6,0,1,0,0
H_L,0,0,5,0,2,18,47,0,4,0,0,0
H_M,0,0,7,0,0,28,3,0,2,0,0,0
L_H,0,0,0,0,0,100,6,12,0,3325,13,1
L_L,0,0,0,0,0,1,1,0,0,13,3,1
L_M,0,0,0,0,0,1,0,2,1,65,0,0
M_H,0,0,0,0,0,151,8,20,0,442,0,0
M_L,0,0,0,0,0,0,28,0,1,8,0,1
M_M,0,0,0,0,0,3,1,0,1,12,0,0


## Combine clustering techniques##
Group 1's magnitude of use * Group 2's shapes

In [40]:
np.sort(clusters.Cluster2.unique())

array(['C_1_L', 'C_1_M', 'C_1_S', 'C_2_L', 'C_2_M', 'C_2_S', 'C_3_M',
       'C_3_S', 'C_4_S', 'R_0', 'R_1', 'R_2'], dtype=object)

In [ ]:
# residential vs commercial flag
clusters['Type'] = 'Commercial'
clusters.loc[(clusters.REVENUCODE == 1), 'Type'] = 'Residential'

clusters['ClusterComb'] = None
if clusters['Type'] == 'Residential':
    clusters.loc[ (clusters.UseGroup == 0) & (clusters.Cluster2 == 'R_0'), 'ClusterComb'] = 'R_No_0' 
    clusters.loc[ (clusters.UseGroup == 1) & (clusters.Cluster2 == 'R_0'), 'ClusterComb'] = 'R_Lo_0' 
    clusters.loc[ (clusters.UseGroup == 2) & (clusters.Cluster2 == 'R_0'), 'ClusterComb'] = 'R_Md_0' 
    clusters.loc[ (clusters.UseGroup == 3) & (clusters.Cluster2 == 'R_0'), 'ClusterComb'] = 'R_Hi_0' 
    clusters.loc[ (clusters.UseGroup == 4) & (clusters.Cluster2 == 'R_0'), 'ClusterComb'] = 'R_VH_0' 
    
    clusters.loc[ (clusters.UseGroup == 0) & (clusters.Cluster2 == 'R_1'), 'ClusterComb'] = 'R_No_1' 
    clusters.loc[ (clusters.UseGroup == 1) & (clusters.Cluster2 == 'R_1'), 'ClusterComb'] = 'R_Lo_1' 
    clusters.loc[ (clusters.UseGroup == 2) & (clusters.Cluster2 == 'R_1'), 'ClusterComb'] = 'R_Md_1' 
    clusters.loc[ (clusters.UseGroup == 3) & (clusters.Cluster2 == 'R_1'), 'ClusterComb'] = 'R_Hi_1' 
    clusters.loc[ (clusters.UseGroup == 4) & (clusters.Cluster2 == 'R_1'), 'ClusterComb'] = 'R_VH_1' 

    clusters.loc[ (clusters.UseGroup == 0) & (clusters.Cluster2 == 'R_2'), 'ClusterComb'] = 'R_No_2' 
    clusters.loc[ (clusters.UseGroup == 1) & (clusters.Cluster2 == 'R_2'), 'ClusterComb'] = 'R_Lo_2' 
    clusters.loc[ (clusters.UseGroup == 2) & (clusters.Cluster2 == 'R_2'), 'ClusterComb'] = 'R_Md_2' 
    clusters.loc[ (clusters.UseGroup == 3) & (clusters.Cluster2 == 'R_2'), 'ClusterComb'] = 'R_Hi_2' 
    clusters.loc[ (clusters.UseGroup == 4) & (clusters.Cluster2 == 'R_2'), 'ClusterComb'] = 'R_VH_2' 

elif clusters['Type'] == 'Commercial':
    # small commercial
    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_S_1'), 'ClusterComb'] = 'CS_No_0' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_S_1'), 'ClusterComb'] = 'CS_Lo_0' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_S_1'), 'ClusterComb'] = 'CS_Md_0' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_S_1'), 'ClusterComb'] = 'CS_Hi_0' 

    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_S_2'), 'ClusterComb'] = 'CS_No_1' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_S_2'), 'ClusterComb'] = 'CS_Lo_1' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_S_2'), 'ClusterComb'] = 'CS_Md_1' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_S_2'), 'ClusterComb'] = 'CS_Hi_1'

    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_S_3'), 'ClusterComb'] = 'CS_No_2' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_S_3'), 'ClusterComb'] = 'CS_Lo_2' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_S_3'), 'ClusterComb'] = 'CS_Md_2' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_S_3'), 'ClusterComb'] = 'CS_Hi_2'
    
    # med commercial
    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_M_1'), 'ClusterComb'] = 'CM_No_0' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_M_1'), 'ClusterComb'] = 'CM_Lo_0' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_M_1'), 'ClusterComb'] = 'CM_Md_0' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_M_1'), 'ClusterComb'] = 'CM_Hi_0' 

    clusters.loc[ (clusters.UseGroup == 1) & (clusters.Cluster2 == 'C_M_2'), 'ClusterComb'] = 'CM_No_1' 
    clusters.loc[ (clusters.UseGroup == 2) & (clusters.Cluster2 == 'C_M_2'), 'ClusterComb'] = 'CM_Lo_1' 
    clusters.loc[ (clusters.UseGroup == 3) & (clusters.Cluster2 == 'C_M_2'), 'ClusterComb'] = 'CM_Md_1' 
    clusters.loc[ (clusters.UseGroup == 4) & (clusters.Cluster2 == 'C_M_2'), 'ClusterComb'] = 'CM_Hi_1'

    clusters.loc[ (clusters.UseGroup == 1) & (clusters.Cluster2 == 'C_M_3'), 'ClusterComb'] = 'CM_No_2' 
    clusters.loc[ (clusters.UseGroup == 2) & (clusters.Cluster2 == 'C_M_3'), 'ClusterComb'] = 'CM_Lo_2' 
    clusters.loc[ (clusters.UseGroup == 3) & (clusters.Cluster2 == 'C_M_3'), 'ClusterComb'] = 'CM_Md_2' 
    clusters.loc[ (clusters.UseGroup == 4) & (clusters.Cluster2 == 'C_M_3'), 'ClusterComb'] = 'CM_Hi_2'
    
    # large commercial
    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_L_1'), 'ClusterComb'] = 'CL_No_0' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_L_1'), 'ClusterComb'] = 'CL_Lo_0' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_L_1'), 'ClusterComb'] = 'CL_Md_0' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_L_1'), 'ClusterComb'] = 'CL_Hi_0' 

    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_L_2'), 'ClusterComb'] = 'CL_No_1' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_L_2'), 'ClusterComb'] = 'CL_Lo_1' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_L_2'), 'ClusterComb'] = 'CL_Md_1' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_L_2'), 'ClusterComb'] = 'CL_Hi_1'

    clusters.loc[ (clusters.UseGroup == 1) & 
                 (clusters.Cluster2 == 'C_L_3'), 'ClusterComb'] = 'CL_No_2' 
    clusters.loc[ (clusters.UseGroup == 2) & 
                 (clusters.Cluster2 == 'C_L_3'), 'ClusterComb'] = 'CL_Lo_2' 
    clusters.loc[ (clusters.UseGroup == 3) & 
                 (clusters.Cluster2 == 'C_L_3'), 'ClusterComb'] = 'CL_Md_2' 
    clusters.loc[ (clusters.UseGroup == 4) & 
                 (clusters.Cluster2 == 'C_L_3'), 'ClusterComb'] = 'CL_Hi_2'

In [141]:
clusters.to_pickle(location+'codebook_clusters_combined.pkl.zip')

## Old/for reference ##

In [4]:
# consistent col name 
group_1.rename(columns={'Cluster':'group1_cluster'}, inplace=True)

group_2_test_CW.rename(columns={'DMeterNo':'ID', 'Lable': 'group2_cluster_W'}, inplace=True)
group_2_test_RW.rename(columns={'DMeterNo':'ID', 'Lable': 'group2_cluster_W'}, inplace=True)

# group_2_test_C.rename(columns={'DMeterNo':'ID', 'Cluster': 'group2_cluster'}, inplace=True)
# group_2_test_R.rename(columns={'DMeterNo':'ID', 'Cluster': 'group2_cluster'}, inplace=True)

In [47]:
group_2_CW.head()

,DMeterNo,AVG,Lable,Size,Cluster
0,2575037524724,5412.083333,1,L,C1L
1,2574628679116,3267.416667,1,L,C1L
2,3881639196592,3041.083333,1,L,C1L
3,2769160671848,1912.500000,1,L,C1L
4,1721259066372,1345.000000,1,L,C1L


In [16]:
group_2_test_CW['Lable'].unique()

array([1, 2, 3, 4])

In [5]:
# consistent col type
# group_2_test_C = group_2_test_C.astype({"ID": object})
# group_2_test_R = group_2_test_R.astype({"ID": object})

# group_2_test_CW = group_2_test_CW.astype({"ID": object})
# group_2_test_RW = group_2_test_RW.astype({"ID": object})

In [6]:
# split group1's data to commerce and resident
group_1_test_C = group_1[(group_1['cTARIFF'] == 'ComH')]
group_1_test_R = group_1[(group_1['cTARIFF'] == 'ResH')]

In [7]:
# explore the data

print('group1 commerce data scope: %s' % (group_1_test_C.shape,))
print('group2 full commerce data scope: %s' % (group_2_test_C.shape,))
print('group2 winter commerce data scope: %s' % (group_2_test_CW.shape,))


print('group1 resident data scope: %s' % (group_1_test_R.shape,))
print('group2 full resident data scope: %s' % (group_2_test_R.shape,))
print('group2 winter resident data scope: %s' % (group_2_test_RW.shape,))


group1 commerce data scope: (666, 8)
group2 full commerce data scope: (825, 366)
group2 winter commerce data scope: (825, 124)
group1 resident data scope: (5083, 8)
group2 full resident data scope: (6218, 368)
group2 winter resident data scope: (5593, 122)


In [8]:
print(group_1_test_C.ID.nunique())
print(group_2_test_C.ID.nunique())
print(group_2_test_CW.ID.nunique())

print(group_1_test_R.ID.nunique())
print(group_2_test_R.ID.nunique())
print(group_2_test_RW.ID.nunique())


664
821
821
5052
6047
5472


In [9]:
# remove duplicates

group_1_C = group_1_test_C.drop_duplicates("ID") 
group_2_C = group_2_test_C.drop_duplicates("ID") 
group_2_CW = group_2_test_CW.drop_duplicates("ID")


group_1_R = group_1_test_R.drop_duplicates("ID") 
group_2_R = group_2_test_R.drop_duplicates("ID") 
group_2_RW = group_2_test_RW.drop_duplicates("ID") 

In [10]:
# compare the data before and after removing duplicates


print('old - group1 commerce data scope: %s' % (group_1_test_C.shape,))
print('new - group1 commerce data scope: %s' % (group_1_C.shape,))

print('old - group2 full commerce data scope: %s' % (group_2_test_C.shape,))
print('new - group2 full commerce data scope: %s' % (group_2_C.shape,))

print('old - group2 winter commerce data scope: %s' % (group_2_test_CW.shape,))
print('new - group2 winter commerce data scope: %s' % (group_2_CW.shape,))


print('old - group1 resident data scope: %s' % (group_1_test_R.shape,))
print('new - group1 resident data scope: %s' % (group_1_R.shape,))

print('old - group2 full resident data scope: %s' % (group_2_test_R.shape,))
print('new - group2 full resident data scope: %s' % (group_2_R.shape,))

print('old - group2 winter resident data scope: %s' % (group_2_test_RW.shape,))
print('new - group2 winter resident data scope: %s' % (group_2_RW.shape,))


old - group1 commerce data scope: (666, 8)
new - group1 commerce data scope: (664, 8)
old - group2 full commerce data scope: (825, 366)
new - group2 full commerce data scope: (821, 366)
old - group2 winter commerce data scope: (825, 124)
new - group2 winter commerce data scope: (821, 124)
old - group1 resident data scope: (5083, 8)
new - group1 resident data scope: (5052, 8)
old - group2 full resident data scope: (6218, 368)
new - group2 full resident data scope: (6047, 368)
old - group2 winter resident data scope: (5593, 122)
new - group2 winter resident data scope: (5472, 122)


In [11]:
# crosstabs

print('comm - full year')
print(pd.crosstab(group_1_C.group1_cluster, group_2_C.group2_cluster))


print('comm - winter')
print(pd.crosstab(group_1_C.group1_cluster, group_2_CW.group2_cluster_W))


print('res - full year')
print(pd.crosstab(group_1_R.group1_cluster, group_2_C.group2_cluster))

print('res - winter')
print(pd.crosstab(group_1_R.group1_cluster, group_2_CW.group2_cluster_W))



comm - full year
group2_cluster  0  1   2
group1_cluster          
H_H             6  8  28
H_L             0  0   3
H_M             1  1   2
L_H             2  1  16
L_L             0  0   1
M_H             1  5  21
M_L             1  0   1
N_H             0  3   5
N_L             1  1   2
comm - winter
group2_cluster_W   1  2  3  4
group1_cluster               
H_H               32  4  3  4
H_L                2  1  0  0
H_M                1  2  0  1
L_H               14  1  3  1
L_L                1  0  0  0
M_H               22  3  1  0
M_L                2  1  0  0
N_H                8  0  0  0
N_L                3  1  0  0
res - full year
group2_cluster   0   1    2
group1_cluster             
L_H             56  41  319
L_L              0   0    1
L_M              0   0   11
M_H              6   6   41
M_L              1   0    2
N_H             17  12   74
N_L              2   4   10
N_M              1   4   18
res - winter
group2_cluster_W    1   2   3   4
group1_cluster       